## Gaussian Mixture Models

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [2]:
mu1 = -1
mu2 = 3
sig1 = 0.5
sig2 = 1
N = 100
np.random.seed(10)
x11=np.random.randn(N,1)*sig1 + mu1
x12=np.random.randn(N,1)*sig1 + mu1+3
x21=np.random.randn(N,1)*sig2 + mu2
x22=np.random.randn(N,1)*sig2 + mu2+3
c = np.vstack((np.zeros((N,1)), np.ones((N,1))))
x1 = np.hstack((x11,x12))
x2 = np.hstack((x21,x22))

X = np.hstack( (np.vstack( (x1,x2) ),c) )
np.random.shuffle(X)
dataset = pd.DataFrame(data=X, columns=['x','y','c'])

In [3]:
dataset

,x,y,c
0,-0.334207,2.058738,0.0
1,3.264395,5.636814,1.0
2,-0.669884,1.856185,0.0
3,1.995284,4.484924,1.0
4,3.588041,4.739172,1.0
...,...,...,...
195,-1.060953,1.742545,0.0
196,-1.365985,2.723583,0.0
197,-1.133659,1.885027,0.0
198,3.602022,5.744720,1.0


In [12]:
class GaussianMixtureModel:
    def __init__(self,
                 num_clusters: int = 1,
                 tolerance: float = 1e-5,
                 num_iters: int = 10,
                 ):
        self.num_clusters = num_clusters
        self.tolerance: tolerance
        self.num_iters = num_iters
        self.X = None
        
    def gaussian(self, mu, cov):
        n, d = self.X.shape
        diff = (self.X - mu).T
        gauss = np.diagonal(1 / ((2 * np.pi) ** (n / 2) * np.linalg.det(cov) ** 0.5) * np.exp(-0.5 * np.dot(np.dot(diff.T, np.linalg.inv(cov)), diff))).reshape(-1, 1)
        return gauss.squeeze()

    def initial_means_covs(self):
        n, d = self.X.shape
        # Mean
        random_idx = np.random.permutation(n)
        selected_index = random_idx[0:self.num_clusters]
        Xmean_init = self.X[selected_index]
        # Covs
        covs = np.zeros((self.num_clusters, d, d))
        for cluster in range(self.num_clusters):
            covs[cluster] = self.Xcov
        return Xmean_init, covs
    
    def compute_Xmean_Xcov(self):
        n = self.X.shape[0]
        Xmean = np.mean(self.X, axis=0)
        Xcentred = self.X - Xmean
        Xcov = (1/n) * (Xcentred.T@Xcentred)
        return Xmean, Xcov
    
    def initialize_phis(self, num_clusters):
        """ Set all component distribution prior estimates 
        to the uniform distribution
        """
        return [1/num_clusters for _ in range(num_clusters)]
    
    def initialize_cluster(self):
        n = X.shape[0]
        cluster_w = np.ones((n, self.num_clusters)) * 1/self.num_clusters
        return cluster_w
    
    def multivariate_pdf(self, cluster: int):
        var = multivariate_normal(mean=self.means[cluster],
                                  cov=self.covs[cluster],
                                  seed=42,
                                 )
        return var.pdf(self.X)
    
    def compute_pXi_Clust_i(self):
        for cluster in range(self.num_clusters):
            self.pXi_Clust_i[:, cluster] = self.gaussian(self.means[cluster],
                                                         self.covs[cluster])
            # self.multivariate_pdf(cluster)
        return None
            
    def compute_SigmaPXi_Zi_n_phi(self):
        return np.sum(self.pXi_Clust_i * self.phis, axis=1)
    
    def compute_phis(self) -> None:
        self.phis = np.mean(self.cluster_weights, axis=0, keepdims=True)
        return None
        
    def compute_means(self) -> None:
        for cluster in range(self.num_clusters): 
            self.means[cluster] = np.sum(self.X * self.cluster_weights[:, cluster].reshape(-1, 1)) \
                                / np.sum(self.cluster_weights[:, cluster])
        return None
#     def comp_conv(self):
#         for j in range(self.X.shape[0]):
#             diff = (self.X[j] - mu_k).reshape(-1, 1)
#             cov_k += gamma_nk[j] * np.dot(diff, diff.T)
            
#         cov_k /= N_k
    def compute_covs(self) -> None:
        for cluster in range(self.num_clusters):
            mean_ = self.means[cluster]
            Xcentred = self.X - mean_
            Xcov = (Xcentred.T@(self.cluster_weights[:, cluster].reshape(-1, 1)*Xcentred))
            self.covs[cluster] = Xcov 
        return None
    
    def compute_likelihood(self) -> float:
        l = self.likelihood + 0.1
        return l
    
    def fit(self, X: pd.DataFrame):
        n, d = X.shape
        self.X = X.values.copy()
        self.Xmean, self.Xcov = self.compute_Xmean_Xcov()
        
        # Initialize mean and covariance randomly
        self.means, self.covs = self.initial_means_covs()        
        
        # Initialize clusters
        self.cluster_weights = self.initialize_cluster()
        self.phis = self.initialize_phis(self.num_clusters)
        
        # Probability distributions
        self.pXi_Clust_i = np.zeros((n, self.num_clusters))
        self.Sigma_PXi_Zi_n_phi = np.zeros(n)
        
        iter = 0
        mean_norm = 1
        self.prev_likelihood = 0
        self.likelihood = 0
        
        while (iter < self.num_iters) or \
              (np.linalg.norm(self.prev_likelihood - self.likelihood) <= self.tolerance):
            # Perform Expectation(E) step
            # Compute probability of data given the cluster
            self.compute_pXi_Clust_i()
            self.compute_SigmaPXi_Zi_n_phi()
            # Compute probability of cluster           
            for cluster in range(self.num_clusters): 
                self.cluster_weights[:, cluster] = self.pXi_Clust_i[:, cluster] \
                                                 * self.phis[cluster] \
                                                 / self.Sigma_PXi_Zi_n_phi
            
            # Perform Maximization(M) Step
            self.compute_phis()
            self.compute_means()
            
            self.compute_covs()
            
            # compute log-likelihood
            self.likelihood = self.compute_likelihood()
            print(f'Log-likelihood for this {iter} iteration is {self.likelihood}')
            self.prev_likelihood = self.likelihood
                                    
            iter += 1

In [ ]:
gmm = GaussianMixtureModel(num_clusters=2)
gmm.fit(dataset[['x', 'y']])

In [ ]:
from scipy.stats import multivariate_normal
var = multivariate_normal(mean=[0,0], cov=[[1,0],[0,1]], seed=42)
var.pdf([[0.0,0.0],
         [0.0, 0.0]])